In [1]:
# pip install duckdb
# pip install jupysql
# pip install notebook
# pip install tabulate
# pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [3]:
# If on windows, this only works if you're running Jupyter as an admin
# pip install duckdb-engine

  Using cached duckdb_engine-0.13.1-py3-none-any.whl (47 kB)
  Using cached packaging-24.1-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 20.9
    Uninstalling packaging-20.9:
      Successfully uninstalled packaging-20.9
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import duckdb
import sqlalchemy
from datetime import date
import json
import pathlib
import importlib
%load_ext sql

In [2]:
# Set up SQL connection
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Change the billboard_observation_date!

In [3]:
current_date = date.today().strftime('%Y-%m-%d')
billboard_observation_date = '2024-07-23'

In [4]:
# Replace with your filepath. Forward slashes work for both Mac and Windows
%sql duckdb:///C:/Users/Admin/billboard/raw_db.duckdb

In [5]:
# Test connection
%sql USE raw_db;
%sql USE raw_db.billboard;
%sql SELECT * FROM raw_db.billboard.artist LIMIT 2;

,artist_id,artist_name,pct_female,last_update
0,1,Zach Bryan,0.0,2024-08-23
1,2,Morgan Wallen,0.0,2024-08-23


# Update upload/rank_by_name

In [6]:
# Update this for this week's rankings in upload/rank_by_name.py
# This is easier in an IDE that allows opt+arrow_key to move lines up and down

In [7]:
# TODO: figure out importlib.reload() here so I don't have to kill the kernel to reload
from upload.rank_by_name import rankings

if len(rankings) != 50:
    print(f'{len(rankings)} rankings, but expected 50')

rankings[:5] + ['...'] + rankings[45:]

['A Bar Song (Tipsy)',
 'I Had Some Help',
 'Lies Lies Lies',
 'Cowgirls',
 'Pink Skies',
 '...',
 "What He'll Never Have",
 'Mechanical Bull',
 'Mama I Lied',
 'Northern Thunder',
 'Relapse']

In [8]:
%%sql df_song_labeled <<
SELECT
    ss.song_id
    , ss.song_name
    , art.artist_name
    , s2a.relationship_type
    , ss.untracked_weeks_on_chart
FROM song ss
LEFT JOIN song_to_artist s2a
    ON ss.song_id = s2a.song_id
LEFT JOIN artist art
    ON s2a.artist_id = art.artist_id
;

In [9]:
df_song_labeled.tail(20)

,song_id,song_name,artist_name,relationship_type,untracked_weeks_on_chart
108,17,Blackbiird,Tanner Adell,Other main artist,0.0
109,17,Blackbiird,Beyonce,Lead artist,0.0
110,50,Whiskey Whiskey,Moneybagg Yo,Lead artist,0.0
111,2,Last Night,None,None,51.0
112,3,Thinkin' Bout Me,None,None,46.0
113,4,Fast Car,None,None,43.0
114,5,White Horse,None,None,26.0
115,6,World On Fire,None,None,32.0
116,7,Need A Favor,None,None,51.0
117,8,Pretty Little Poison,None,None,32.0


In [10]:
existing_song_to_rank = {}
new_song_to_rank = {}
for song, rank_ix in zip(rankings, range(1,51)):
    print(df_song_labeled.loc[df_song_labeled['song_name'] == song].to_markdown())
    song_id = input("Please enter the song_id or 'New Song'\n")

    if song_id == '': # hit enter to take the most common song ID
        song_id = int(
            df_song_labeled.loc[
                df_song_labeled['song_name'] == song
            ]['song_id'].mode()
        )
        
    if str(song_id).lower() == 'new song':
        new_song_to_rank[song] = rank_ix
    else:
        existing_song_to_rank[int(song_id)] = rank_ix

|    |   song_id | song_name          | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:-------------------|:--------------|:--------------------|---------------------------:|
| 16 |        34 | A Bar Song (Tipsy) | Shaboozey     | Lead artist         |                          0 |
Please enter the song_id or 'New Song'

|    |   song_id | song_name       | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:----------------|:--------------|:--------------------|---------------------------:|
| 20 |        38 | I Had Some Help | Morgan Wallen | Other main artist   |                          0 |
| 80 |        38 | I Had Some Help | Post Malone   | Lead artist         |                          0 |
Please enter the song_id or 'New Song'

|    |   song_id | song_name      | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:---------------|:--------------|:--------------------|-------

Please enter the song_id or 'New Song'

|    |   song_id | song_name   | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:------------|:--------------|:--------------------|---------------------------:|
|  7 |        25 | Chevrolet   | Jelly Roll    | Featured artist     |                          1 |
| 77 |        25 | Chevrolet   | Dustin Lynch  | Lead artist         |                          1 |
Please enter the song_id or 'New Song'

|    |   song_id | song_name        | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:-----------------|:--------------|:--------------------|---------------------------:|
| 24 |        42 | Hang Tight Honey | Lainey Wilson | Lead artist         |                          0 |
Please enter the song_id or 'New Song'

|    |   song_id | song_name   | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:------------|:--------------|:----------------

Please enter the song_id or 'New Song'

| song_id   | song_name   | artist_name   | relationship_type   | untracked_weeks_on_chart   |
|-----------|-------------|---------------|---------------------|----------------------------|
Please enter the song_id or 'New Song'
New Song
|    |   song_id | song_name   | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:------------|:--------------|:--------------------|---------------------------:|
| 10 |        28 | Nine Ball   | Zach Bryan    | Lead artist         |                          3 |
Please enter the song_id or 'New Song'

|    |   song_id | song_name             | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:----------------------|:--------------|:--------------------|---------------------------:|
| 13 |        31 | What He'll Never Have | Dylan Scott   | Lead artist         |                          0 |
Please enter the song_id or 'New Song'

| song_id   

In [11]:
print(json.dumps(existing_song_to_rank, indent = 2))
print(json.dumps(new_song_to_rank, indent = 2))

{
  "34": 1,
  "38": 2,
  "58": 3,
  "32": 4,
  "43": 5,
  "57": 6,
  "37": 7,
  "27": 8,
  "47": 9,
  "1": 10,
  "40": 11,
  "52": 12,
  "50": 13,
  "48": 14,
  "53": 15,
  "24": 16,
  "56": 17,
  "21": 18,
  "30": 19,
  "59": 20,
  "72": 21,
  "25": 22,
  "42": 23,
  "60": 24,
  "61": 25,
  "22": 26,
  "39": 27,
  "41": 28,
  "71": 29,
  "62": 30,
  "55": 31,
  "73": 32,
  "92": 33,
  "26": 34,
  "33": 35,
  "75": 36,
  "44": 37,
  "35": 38,
  "23": 39,
  "29": 40,
  "94": 41,
  "36": 42,
  "51": 43,
  "28": 45,
  "31": 46,
  "74": 48,
  "49": 50
}
{
  "Boons": 44,
  "Mechanical Bull": 47,
  "Northern Thunder": 49
}


## Add artists on any new songs that aren't in the artist table yet to its CSV

- Boons: 1
- Mechanical Bull: 1
- Northern Thunder: 1

In [12]:
df_artist = pd.read_csv('source_data/artist.csv')

In [13]:
new_song_artists = [
    'Zach Bryan' # 1
]
df_artist[df_artist['artist_name'].isin(new_song_artists)]

,artist_id,artist_name,pct_female,last_update
0,1,Zach Bryan,0.0,2024-08-23


In [14]:
df_artist['artist_id'].max()

63

In [16]:
df_artist = pd.concat(
    [
        df_artist
        , pd.DataFrame(
            np.array([
                # [62, 'Dylan Marlowe', 0, current_date]
                # , [63, 'Noeline Hofmann', 1, current_date]
            ])
            , columns = ['artist_id', 'artist_name', 'pct_female', 'last_update']
        )
    ]
    , ignore_index = True
)

In [17]:
df_artist.to_csv('source_data/artist.csv', index=False)

## Add all new songs to the song table CSV

In [15]:
df_song_raw = pd.read_csv('source_data/song.csv')

In [16]:
max_song_id = df_song_raw['song_id'].max()
array_new_songs = []
for song_name in [
    'Boons' # 1
    , 'Mechanical Bull' # 1
    , 'Northern Thunder' # 1
]:
    max_song_id += 1
    array_new_songs += [[max_song_id, song_name, 0, current_date]]

array_new_songs

[[95, 'Boons', 0, '2024-09-08'],
 [96, 'Mechanical Bull', 0, '2024-09-08'],
 [97, 'Northern Thunder', 0, '2024-09-08']]

In [17]:
df_song_raw = pd.concat(
    [
        df_song_raw
        , pd.DataFrame(
            np.array(array_new_songs)
            , columns = ['song_id', 'song_name', 'untracked_weeks_on_chart', 'last_update']
        )
    ]
    , ignore_index = True
)

In [18]:
# Update untracked_weeks_on_chart if needed
df_song_raw[-len(array_new_songs)-1:]

,song_id,song_name,untracked_weeks_on_chart,last_update
93,94,Purple Gas,0.0,2024-09-08
94,95,Boons,0,2024-09-08
95,96,Mechanical Bull,0,2024-09-08
96,97,Northern Thunder,0,2024-09-08


In [19]:
# Adjust untracked weeks if they aren't zero
# Example: df_song_raw.loc[92, 'untracked_weeks_on_chart'] = 6

In [20]:
df_song_raw.to_csv('source_data/song.csv', index=False)

## Add all new songs to the song_to_artist table CSV

In [21]:
df_artist[df_artist['artist_name'].isin(new_song_artists)]

,artist_id,artist_name,pct_female,last_update
0,1,Zach Bryan,0.0,2024-08-23


In [22]:
df_s2a_raw = pd.read_csv('source_data/song_to_artist.csv').fillna('')

In [23]:
array_new_s2a = []
for s2a in [ # song_id, artist_id, relationship_conjunction, relationship_index
    [95, 1, '', '']
    , [96, 1, '', '']
    , [97, 1, '', '']
]:
    if s2a[3] == '': # Only lead artists don't have a relationship index
        reln = 'Lead artist'
    elif s2a[2] == '': # Featured artists don't have an explicit relationship conjunction
        reln = 'Featured artist' 
    else:
        reln = 'Other main artist'
        
    array_new_s2a += [[
        '-'.join([str(s2a[0]), reln, str(s2a[3])])
        , s2a[0]
        , s2a[1]
        , reln
        , s2a[2]
        , s2a[3]
        , current_date
    ]]
    
array_new_s2a

[['95-Lead artist-', 95, 1, 'Lead artist', '', '', '2024-09-08'],
 ['96-Lead artist-', 96, 1, 'Lead artist', '', '', '2024-09-08'],
 ['97-Lead artist-', 97, 1, 'Lead artist', '', '', '2024-09-08']]

In [24]:
df_s2a_raw = pd.concat(
    [
        df_s2a_raw
        , pd.DataFrame(
            np.array(array_new_s2a)
            , columns = [
                'song_to_artist_pk'
                , 'song_id'
                , 'artist_id'
                , 'relationship_type'
                , 'relationship_conjunction'
                , 'relationship_index'
                , 'last_update'
            ]
        )
    ]
    , ignore_index = True
)

In [25]:
df_s2a_raw.tail(1)

,song_to_artist_pk,song_id,artist_id,relationship_type,relationship_conjunction,relationship_index,last_update
113,97-Lead artist-,97,1,Lead artist,,,2024-09-08


In [26]:
df_s2a_raw.to_csv('source_data/song_to_artist.csv', index=False)

## Convert new_song_to_rank to the id_to_rankix format of existing_song_to_rank
Then stack the two dicts of songs and insert to CSV with more info like current_date for last_update

In [27]:
all_song_to_rank = existing_song_to_rank.copy()
for song_name, rank in new_song_to_rank.items():
    songs = list(df_song_raw[df_song_raw['song_name'] == song_name]['song_id'])
    
    if len(songs) == 1:
        song_id = songs[0]
    elif len(songs) > 1:
        song_id = int(input(f'{str(songs)}: Choose ID of song actually at rank {rank}\n'))
    else:
        raise Exception(f'The song at rank {rank} is missing from the song CSV')
    
    all_song_to_rank[int(song_id)] = rank

In [28]:
array_new_ranking = [
    [
        '-'.join([billboard_observation_date, str(rank)])
        , billboard_observation_date
        , rank
        , song_id
        , current_date
    ]
    for song_id, rank
    in all_song_to_rank.items()
]

array_new_ranking # Make sure it's 50 rows

[['2024-07-23-1', '2024-07-23', 1, 34, '2024-09-08'],
 ['2024-07-23-2', '2024-07-23', 2, 38, '2024-09-08'],
 ['2024-07-23-3', '2024-07-23', 3, 58, '2024-09-08'],
 ['2024-07-23-4', '2024-07-23', 4, 32, '2024-09-08'],
 ['2024-07-23-5', '2024-07-23', 5, 43, '2024-09-08'],
 ['2024-07-23-6', '2024-07-23', 6, 57, '2024-09-08'],
 ['2024-07-23-7', '2024-07-23', 7, 37, '2024-09-08'],
 ['2024-07-23-8', '2024-07-23', 8, 27, '2024-09-08'],
 ['2024-07-23-9', '2024-07-23', 9, 47, '2024-09-08'],
 ['2024-07-23-10', '2024-07-23', 10, 1, '2024-09-08'],
 ['2024-07-23-11', '2024-07-23', 11, 40, '2024-09-08'],
 ['2024-07-23-12', '2024-07-23', 12, 52, '2024-09-08'],
 ['2024-07-23-13', '2024-07-23', 13, 50, '2024-09-08'],
 ['2024-07-23-14', '2024-07-23', 14, 48, '2024-09-08'],
 ['2024-07-23-15', '2024-07-23', 15, 53, '2024-09-08'],
 ['2024-07-23-16', '2024-07-23', 16, 24, '2024-09-08'],
 ['2024-07-23-17', '2024-07-23', 17, 56, '2024-09-08'],
 ['2024-07-23-18', '2024-07-23', 18, 21, '2024-09-08'],
 ['2024-07-

In [29]:
df_billboard_ranking_raw = pd.concat(
    [
        pd.read_csv('source_data/billboard_ranking.csv').fillna('')
        , pd.DataFrame(
            np.array(array_new_ranking)
            , columns = [
                'billboard_ranking_pk'
                , 'billboard_observation_date'
                , 'billboard_rank'
                , 'song_id'
                , 'last_update'
            ]
        )
    ]
    , ignore_index = True
)

In [30]:
df_billboard_ranking_raw.to_csv('source_data/billboard_ranking.csv', index=False)

## Push to duckdb

In [31]:
# TODO: instead of purging and rebuilding, use the raw data to run updates
%sql --close duckdb:///C:/Users/Admin/billboard/raw_db.duckdb --quiet
pathlib.Path('raw_db.duckdb').unlink(missing_ok = True)

TODO: integrate into jupyter using `%python`

In a command terminal
- `python table_configs/run_schema_configs.py billboard_schema.sql`
- `python table_configs/run_table_configs.py`
- `python main.py`

## Run the ranking SQL script

In [32]:
with duckdb.connect('raw_db.duckdb') as conn:
    with open('analysis/artist_ranking.sql') as sql_file:
        artist_ranking = conn.sql(sql_file.read())
    
    # TODO: Grab headers from the SQL too
    df_artist_ranking = pd.DataFrame(artist_ranking.fetchall())

In [33]:
df_artist_ranking

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,Zach Bryan,56,56,56,92,92,92,15,15,15,15,15,15,5.0,5.0,2024-07-23
1,15,Post Malone,45,45,45,45,45,45,18,18,18,18,18,18,2.0,2.0,2024-07-23
2,2,Morgan Wallen,14,21,28,98,105,112,2,3,4,2,3,4,2.0,2.0,2024-07-23
3,5,Jelly Roll,11,16,25,11,16,32,3,4,6,3,4,6,10.0,10.0,2024-07-23
4,4,Megan Moroney,14,14,14,14,14,14,3,3,3,3,3,3,21.0,21.0,2024-07-23
5,16,Koe Wetzel,14,14,14,14,14,14,2,2,2,2,2,2,9.0,9.0,2024-07-23
6,6,Luke Combs,12,12,19,47,47,54,2,2,4,2,2,4,3.0,3.0,2024-07-23
7,11,Lainey Wilson,11,11,13,11,46,48,2,2,3,2,2,3,19.0,19.0,2024-07-23
8,8,Bailey Zimmerman,10,10,10,10,10,10,2,2,2,2,2,2,28.0,28.0,2024-07-23
9,25,Shaboozey,9,9,9,9,9,9,2,2,2,2,2,2,1.0,1.0,2024-07-23


In [34]:
with duckdb.connect('raw_db.duckdb') as conn:
    print(conn.sql(
        """
        SELECT billboard_observation_date, COUNT(1) n
        FROM raw_db.billboard.billboard_ranking
        GROUP BY 1
        ORDER BY 1 ASC
        """
    ))

┌────────────────────────────┬───────┐
│ billboard_observation_date │   n   │
│            date            │ int64 │
├────────────────────────────┼───────┤
│ 2024-07-23                 │    50 │
│ 2024-07-31                 │    50 │
│ 2024-08-07                 │    50 │
│ 2024-08-13                 │    50 │
│ 2024-08-23                 │    50 │
│ 2024-08-27                 │    50 │
│ 2024-09-05                 │    50 │
└────────────────────────────┴───────┘

